# Variable Elimination Bayes Network

Primero para empezar a implementar este concepto de redes bayesianas tenemos que tener el mismo por lo que acontinuacion modelelaremos la siguiente red ejemplo de estudiante. 

![title](example.png)

ahora con este ejemplo, lo primero que vamos a crear es hacer nuestro modelado en el software de modelacion 
SamIam que tu puedes ver en este link: http://reasoning.cs.ucla.edu/samiam/help/ asi que para esto primero 
vemos la documentación y modelamos nuestra Bayes Network con java samlam, aqui abajo tenemos el modelo el 
modelado asi como el resultados mostrado 
    
![modeling](sam_example.png)

de acuerdo a la imagen anterior podemos realizar ahora el alcance de la inferencia en nuestra Bayes Network,
para hacer esto primero necesitamos entender la inferencia de acuerdo a la definicion tenemos:

$$P(Y|Z) = \frac{(Y,Z)}{P(Z)}$$

ahora para desarrollar la inferencia bayesianda tenemos que tomar encuenta que  $$Y \in X \quad Z \in X $$ para todo $$X = X_1, X_2, \dots , X_n$$

entonces para especificar mejor y para nuestro ejemplo hallaremos la probabilidad de $$P(L|d,s) = \alpha P(L,d,s)$$ denotando que d: como la dificultada d= True y s: el exmane SAT tambien s = true. asi que para este ejemplo en particular modelando en `samIam`, tenemos el siguiente grafico y valor como respuesta


![modeling](query1.png)
    
como podemos aprecias en el grafico anterior, tenemos que la carta de recomendacion en este caso tiene un 54.16% de posibilidad de aceptacion, debido a que el examen nacional SAT tambien es de suma importancia y que los cursos hayan sido dificils influye por ello tambien se tiene un 45.84% de que no se le entrege al alumno. Bueno despues de todo esta explicacion acontinuacion vamos a pasar a implementar el algoritmo que desarrolla todo este query1. asi que vamos manos a la obra.
- 54.16 True
- 45.84 False

## implementacion BN 

In [7]:
# Implementacion de la clase
# donde se definen las variables de para la Red Bayesiana

class Variable:

    def __init__(self, name, domain=[]):
        
        # creamos las variables del objeto, especiificamos su nombre, dominio, evidencia
        # y ademas el indice asignado
        # nombre, copiar y pasar al dominio, ev
                
        self.name = name     
        self.dom = list(domain)         
        self.evidence_index = 0         
        self.assignment_index = 0                         
                                        

    def add_domain_values(self, values):
        # agregar valores al dominio que devuelven una lista
        for val in values: self.dom.append(val)

    #retorna el indice del dominio de una variable
    def value_index(self, value):
        return self.dom.index(value)
    
    #tamaño del dominio
    def domain_size(self):
        return(len(self.dom))
    
    # variable del dominio
    def domain(self):
        return(list(self.dom))
    
    # modifica la variable como evidencia
    def set_evidence(self,val):
        self.evidence_index = self.value_index(val)

    def get_evidence(self):
        return(self.dom[self.evidence_index])

    # modifica la variable y agregar el valor buscado
    def set_assignment(self, val):
        self.assignment_index = self.value_index(val)

    def get_assignment(self):
        return(self.dom[self.assignment_index])

    ## estas funcione son especiales de bajo nivel usados
    ## por el objeto factor
    
    # factor objeto
    def set_assignment_index(self, index):
        self.assignment_index = index
        
    #ojb
    def get_assignment_index(self):
        return(self.assignment_index)

    # retornar string de obj
    def __repr__(self):
        return("{}".format(self.name))
    
    #string de ojb2
    def __str__(self):
        return("{}, Dom = {}".format(self.name, self.dom))

# clase factor con sus operaciones
class Factor:

    def __init__(self, name, scope):
        #crea el factor con nombre y la lista de salida 
        self.scope = list(scope)
        self.name = name
        size = 1
        for v in scope:
            size = size * v.domain_size()
        self.values = [0]*size  # valor inicial es uns lista de ceros

    # retorna copia d salidas
    def get_scope(self):
        return list(self.scope)
    
    # inicializar el factor
    def add_values(self, values):
        for t in values:
            index = 0
            for v in self.scope:
                index = index * v.domain_size() + v.value_index(t[0])
                t = t[1:]
            self.values[index] = t[0]
    
    # agregar el valor actual 
    def add_value_at_current_assignment(self, number):
        index = 0
        for v in self.scope:
            index = index * v.domain_size() + v.get_assignment_index()
        self.values[index] = number

    # get valores del factor
    def get_value(self, variable_values):
        index = 0
        for v in self.scope:
            index = index * v.domain_size() + v.value_index(variable_values[0])
            variable_values = variable_values[1:]
        return self.values[index]

    # devolver los valores actuales de factor cuando se asigna
    def get_value_at_current_assignments(self):
        index = 0
        for v in self.scope:
            index = index * v.domain_size() + v.get_assignment_index()
        return self.values[index]

    def print_table(self):
        # imprimir el factor
        saved_values = []  #guardar y return valor asignasdo
        for v in self.scope:
            saved_values.append(v.get_assignment_index())

        self.recursive_print_values(self.scope)

        for v in self.scope:
            v.set_assignment_index(saved_values[0])
            saved_values = saved_values[1:]

    def recursive_print_values(self, vars):
        if len(vars) == 0:
            print("[",end=""),
            for v in self.scope:
                print("{} = {},".format(v.name, v.get_assignment()), end="")
            print("] = {}".format(self.get_value_at_current_assignments()))
        else:
            for val in vars[0].domain():
                vars[0].set_assignment(val)
                self.recursive_print_values(vars[1:])

    def __repr__(self):
        return("{}({})".format(self.name, list(map(lambda x: x.name, self.scope))))

    
# clase para definir red bayesiana
# esta variable tambien guarda los pasos de la variables scopes
# se los factores
    
class BN:

    def __init__(self, name, Vars, Factors):
        self.name = name
        self.Variables = list(Vars)
        self.Factors = list(Factors)
        for f in self.Factors:
            for v in f.get_scope():
                if not v in self.Variables:
                    print(" inicializacion de error de Bayes net")
                    print(" Factor scope {} tiene var {} ", end='')
                    print("  no aparece en la lista de vars {}.".format(list(map(lambda x: x.name, f.get_scope())), v.name, list(map(lambda x: x.name, Vars))))

    def factors(self):
        return list(self.Factors)

    def variables(self):
        return list(self.Variables)

# Multiplicacion de factores
def multiply_factors(Factors):
    # nombre de factor
    newFactorName = ""
    for i in range(len(Factors)):
        cur_factor = Factors[i]
        if(i == 0):
            newFactorName += cur_factor.name
            continue
        if(i == len(Factors) - 1):
            newFactorName = newFactorName + "_" + cur_factor.name
            continue
        else:
            newFactorName = newFactorName + "_" + cur_factor.name + "_"
            continue

    # get de nuevos scope del factors
    newFactorScope = list()
    for factor in Factors:
        for var in factor.get_scope():
            if(var in newFactorScope):
                continue
            newFactorScope.append(var)


    # crear nuevo factor
    newFactor = Factor(newFactorName, newFactorScope)

    # modificar los valores actuales asignados al nuevo factor
    rec_multiply_factors(Factors, newFactorScope, newFactor)

    return newFactor

def rec_multiply_factors(Factors, newFactorScope, newFactor, total=1):
    
    if(not(len(newFactorScope))):
        for factor in Factors:
            total *= factor.get_value_at_current_assignments()
        newFactor.add_value_at_current_assignment(total)
    else:
        for var_val in newFactorScope[0].domain():
            newFactorScope[0].set_assignment(var_val)
            rec_multiply_factors(Factors, newFactorScope[1:], newFactor, total)


# f: factor var: variable y value :domino o nombre de columna
def restrict_factor(f, var, value):

    # cambiar el nombre del factor y scope
    newFactorName = f.name + "_{R: " + str(var.name) + " = " + str(value) + "}"
    newFactorScope = f.get_scope()

    # cambiar  al asigancion a la restriction para eliminar
    # los valores del scope
    var.set_assignment(value)
    newFactorScope.remove(var)
    #nuevo factor
    newFactor = Factor(newFactorName, newFactorScope)

    # Restriccion de factores
    rec_restrict_factor(f, newFactorScope, newFactor)
    return newFactor



def rec_restrict_factor(oldFactor, newFactorScope, newFactor):
    
    if not(newFactorScope):
        old_assign = oldFactor.get_value_at_current_assignments()
        newFactor.add_value_at_current_assignment(old_assign)
    else:
        for var_val in newFactorScope[0].domain():
            newFactorScope[0].set_assignment(var_val)
            rec_restrict_factor(oldFactor, newFactorScope[1:], newFactor)

##
# funcion que devuelve la suma de las variables
# una vez multiplicado los factores y creado un nuevo factor

def sum_out_variable(f, var):
    # cambiar nombre y scope
    newFactorName = f.name + "_{S: " + str(var.name) + "}"
    newFactorScope = f.get_scope()

    # Eliminar vars del scope del nuevo factor
    newFactorScope.remove(var)
    newFactor = Factor(newFactorName, newFactorScope)
    #summar
    rec_sumout_vars(newFactor, f, newFactorScope, var)
    return newFactor

def rec_sumout_vars(newFactor, f, newFactorScope, variable, summ=0):

    if(not(len(newFactorScope))):
        for val in variable.domain():
            variable.set_assignment(val)
            curr = f.get_value_at_current_assignments()
            summ += curr
        newFactor.add_value_at_current_assignment(summ)
    else:
        for var_val in newFactorScope[0].domain():
            newFactorScope[0].set_assignment(var_val)
            rec_sumout_vars(newFactor, f, newFactorScope[1:], variable)


## ORDENACION

def min_fill_ordering(Factors, QueryVar):
    scopes = []
    for f in Factors:
        scopes.append(list(f.get_scope()))
    Vars = []
    for s in scopes:
        for v in s:
            if not v in Vars and v != QueryVar:
                Vars.append(v)

    ordering = []
    while Vars:
        (var,new_scope) = min_fill_var(scopes,Vars)
        ordering.append(var)
        if var in Vars:
            Vars.remove(var)
        scopes = remove_var(var, new_scope, scopes)
    return ordering

##
# FUNCION devuelve lista de scopes 
#
def min_fill_var(scopes, Vars):
    minv = Vars[0]
    (minfill,min_new_scope) = compute_fill(scopes,Vars[0])
    for v in Vars[1:]:
        (fill, new_scope) = compute_fill(scopes, v)
        if fill < minfill:
            minv = v
            minfill = fill
            min_new_scope = new_scope
    return (minv, min_new_scope)

def compute_fill(scopes, var):
    union = []
    for s in scopes:
        if var in s:
            for v in s:
                if not v in union:
                    union.append(v)
    if var in union: union.remove(var)
    return (len(union), union)

# funcion que cambia los scopes que incrementna la eliminacion de vars
def remove_var(var, new_scope, scopes):
    new_scopes = []
    for s in scopes:
        if not var in s:
            new_scopes.append(s)
    new_scopes.append(new_scope)
    return new_scopes

###
def step_1(NetFactors, EvidenceVars):

    for i in range(len(NetFactors)):
        for e_v in EvidenceVars:
            if e_v in NetFactors[i].get_scope():
                newFactor = restrict_factor(NetFactors[i], e_v, e_v.get_evidence())

                # remplazar cada factor en la redfactor con su restrisccion f_{E=e_v}
                NetFactors[i] = newFactor

def step_2(NetFactors, QueryVar):

    for z_j in min_fill_ordering(NetFactors, QueryVar):
        #paso 2.A)
        # encontrar factor f_1, ..., f_k s.t. z_j en scope(f_i)
        f_w_z_j = list()
        for factor in NetFactors:
            if z_j in factor.get_scope():
                f_w_z_j.append(factor)

        # multiplicar los factores q incluyen z_j
        prod_of_f_i = multiply_factors(f_w_z_j)

        #sumar sobre z_j
        g_j = sum_out_variable(prod_of_f_i, z_j)

        #STEP 2.B)
        #eliminar los factores en z_j
        for factor in f_w_z_j:
            NetFactors.remove(factor)

        # agreagar nuevo factor g_j
        NetFactors.append(g_j)

def step_3(NetFactors):

    # toma los factoers con Q
    remaining_factors_w_Q = multiply_factors(NetFactors)

    # normalizacion del factor
    normalization_factor = 0
    for i in remaining_factors_w_Q.values:
        normalization_factor += i

    # comprobar la division por cero
    if(not(normalization_factor)):
        raise ZeroDivisionError

    # get lists de distribucion posterior
    list_of_pos = list()
    for i in remaining_factors_w_Q.values:
        list_of_pos.append(i/normalization_factor)

    return list_of_pos


def VE(Net, QueryVar, EvidenceVars):
    
    NetFactors = Net.factors()

    step_1(NetFactors, EvidenceVars)

    step_2(NetFactors, QueryVar)

    return step_3(NetFactors)


 ### preuba de nuestra consulta 
ahora vamos a probar para el `query1` de nuestro ejemplo

In [8]:
#importamos bnet de import*
import itertools

#La función auxiliar para reordenar el alcance de un factor --- produce un nuevo factor.
#depende de una variable el los nombres de los scopes.
#generar un nuevo factor equivalente para el funcion de entrada de factor f pero con
# el scope ordenado de acuerdo a las lista de "scope_names"

def reorder_factor_scope(f, scope_names):
    # obtener las variables en orden de scope_names
    d = dict()
    vdoms = []
    nscope = []
    for v in f.get_scope():
        d[v.name] = v
        vdoms.append(v.domain())
    for name in scope_names:
        nscope.append(d[name])
    nfactor = Factor(f.name, nscope)
    for t in itertools.product(*vdoms):
        for i, var in enumerate(f.get_scope()):
            var.set_assignment(t[i])
        nfactor.add_value_at_current_assignment(f.get_value_at_current_assignments())
    return nfactor

# funcion que ayuda para probar la calidad de 2 scopes dado las variables en la 
# la lista de scopes quiza sea ordenada diferentemente

def scopesEquiv(s1, s2):
    # s1 y s2 son listas de variables
    return set(s1) == set(s2)

# la diferencia maxima permite la equivalencia de valores
epsilon = 0.0001

#funcion que te ayuda para probar.  Compara la probabilidades 
# de tablas de un factor contra los resultados esperados
#devolver 2 elmentos de lista. Elemento 0  es el numero de los que no coenciden. 
# el elmento 1 son los string con los no conecidentes

def comparetable(factor, values):
    outstring = ""
    counter = 0
    for i in range(0,len(values)):

        if(abs(factor.values[i] - values[i]) > epsilon):
            counter += 1
            outstring += derankmismatch(factor,i) + " : Valor esperado={} Valor actual={}".format(values[i],factor.values[i]) + '\n'

    return [counter, outstring]

# funcion ayuda
def derankmismatch(factor, index):

    outstr = "P("
    value = index
    scope = factor.get_scope()
    variableindex = []
    for v in reversed(scope):
        variableindex.append(int(value % v.domain_size()))
        value = int(value / v.domain_size())

    variableindex = list(reversed(variableindex))
    for i in range(0, len(scope)):
        outstr += ("{} = {},".format(scope[i].name, scope[i].dom[variableindex[i]]))

    return outstr[:-1] + ")"


# funcion para la preuba books no intentos
def scopefilter(scope):
    ret = []
    for i in scope:
        ret.append(repr(i))
    return ret

# Clase Factor preuba de Restriccion 
class RestrictionTest:

    # el factor restringe  con variable = value
    def __init__(self, factor, variable, value, answer, points, name="factor restriction test"):
        self.name = name
        self.factor = factor
        self.variable = variable
        self.value = value
        self.answer = answer
        self.points = points


    def test(self):
       # print("\nPrueba de Ejecuc : {}".format(self.name))
        mark = 0
        result = restrict_factor(self.factor, self.variable, self.value)
        scopetest = scopesEquiv(self.answer[0],scopefilter(result.get_scope()))
        if(scopetest):
            print("\t el scope del factor resultante coencide con el esperado")
        else:
            print("\t[!] Scope del factor resultante no coencide con el esperado : ")
            print("\t\tesperado: " + repr(self.answer[0]))
            print("\t\tActual:  " + repr(scopefilter(result.get_scope())))
        if(scopetest):
            result = reorder_factor_scope(result, self.answer[0])
            tabletest = comparetable(result, self.answer[1])
            copytest = id(result) != id(self.factor)
            if(tabletest[0] == 0):
                print("\t  los valores del factor coencide con el esperado en todos los casos")
                if copytest:
                    mark = self.points
                    print("\t[+] [{}/{}]".format(mark, self.points))
                else:
                    print("\t  el input del factor cambio")
            else:
                print("\t[!] los valores del Factor se pierden con el esperado en {} lugar{}:\n".format(tabletest[0], "" if (tabletest[0] == 1) else "s"))
                print(tabletest[1])
        return mark, self.points


# VARIABLE Factor  suma de la clase test
class SummationTest:
    # el factor es el sumado sobre este,var es la suma de vars
    def __init__(self, factor, variable, answer, points, name="suma salida test de valores"):
        self.name = name
        self.factor = factor
        self.variable = variable
        self.answer = answer
        self.points = points

    def test(self):
       # print("\nPrueba de ejecucion : {}".format(self.name))
        result = sum_out_variable(self.factor, self.variable)
        scopetest = scopesEquiv(self.answer[0],scopefilter(result.get_scope()))
        mark = 0
        if(scopetest):
            print("\t  el scope del factor resultante coencide con el esperado")
        else:
            print("\t[!] Scope del factor resultante no coencide con el esperado : ")
            print("\t\tEsperado: " + repr(self.answer[0]))
            print("\t\tActual:  " + repr(scopefilter(result.get_scope())))

        if(scopetest):
            result = reorder_factor_scope(result, self.answer[0])
            tabletest = comparetable(result, self.answer[1])
            copytest = id(result) != id(self.factor)
            if(tabletest[0] == 0):
                print("\t los valores del factor coencide con el esperado en todos los casos")
                if(copytest):
                    mark = self.points
                    print("\t[+] [{}/{}]".format(mark, self.points))
                else:
                    print("\t el input del factor cambio")
            else:
                print("\t[!] los valores del Factor se pierden con el esperado en {} lugar{}:\n".format(tabletest[0], "" if (tabletest[0] == 1) else "s"))
                print(tabletest[1])

        return mark, self.points

# Clase Mulitiplicaciond e factores
class MultiplyTest:
    # lista de fatores para multiplicar
    def __init__(self, factors, answer,points, name="test d factor de multiplicacion"):
        self.name = name
        self.factors = factors
        self.answer = answer
        self.points = points


    def test(self):
       # print("\nPrueba de ejecucion : {}".format(self.name))
        mark = 0
        result = multiply_factors(self.factors)
        scopetest = scopesEquiv(self.answer[0],scopefilter(result.get_scope()))
        if(scopetest):
            print("\t[+] el scope del factor resultante coencide con el esperado")
        else:
            print("\t[!]  Scope del factor resultante no coencide con el esperado : ")
            print("\t\tEsperado: " + repr(self.answer[0]))
            print("\t\tActual:  " + repr(scopefilter(result.get_scope())))

        if(scopetest):
            result = reorder_factor_scope(result, self.answer[0])
            tabletest = comparetable(result, self.answer[1])
            if(tabletest[0] == 0):
                print("\t los valores del factor coencide con el esperado en todos los casos")
                copytest = id(result) != id(self.factors)
                if(copytest):
                    mark = self.points
                    print("\t[+] [{}/{}]".format(mark, self.points))
                else:
                    print("\t el input del factor cambio")
            else:
                print("\t[!] los valores del Factor se pierden con el esperado en {} lugar{}:\n".format(tabletest[0], "" if (tabletest[0] == 1) else "s"))
                print(tabletest[1])

        return mark, self.points

# Clase Principal Variable de Eliminacion
class VETest:

    def __init__(self, net, evidence, queryVariable, answer, points, name="variable elimination test"):
        self.name = name
        self.net = net
        self.evidence = evidence
        self.queryVariable = queryVariable
        self.answer = answer
        self.points = points

        self.evidenceVars = []

        for i in self.evidence:
            i[0].set_evidence(i[1])
            self.evidenceVars.append(i[0])

    def test(self):
        mark = 0
        for i in self.evidence:
            i[0].set_evidence(i[1])
        result = VE(self.net, self.queryVariable, self.evidenceVars)

        querytest = True

        for i in range(0,len(self.answer[0])):
            if( abs(self.answer[0][i] - result[i]) > epsilon):
                querytest = False

        print("EJECUTAR PRUEBA : {}".format(self.name))
        print("-----------------------------------------------------------")
        print("")
        if(querytest):
            print("Valor SamIam [F,T]: " + repr(self.answer[0]))
            print("Valor Implementacion [F,T]:  " + repr(result))
            mark = self.points
        else:
            print("Valor samiam: " + repr(self.answer[0]))
            print("Valor implementacion:  " + repr(result))

        return mark, self.points

if __name__ == '__main__':

    
    # RED BAYESIANA STUDENT USA
    #-------------------------------------------------------------------------------
    
    #ingresamos primero los campos y sus respectivos valor True,false
    # de todos los nodos  en este caso 5
    Difficulty = Variable('Difficulty', ['True', 'False'])
    Intelligence = Variable('Intelligence', ['True', 'False'])
    Grade = Variable('Grade', ['A', 'B', 'C'])
    SAT = Variable('SAT', ['True', 'False'])
    Letter = Variable('Letter', ['True', 'False'])
    # DIFICULTAD
    F0 = Factor("F0", [Difficulty])
    F0.add_values([['True',  0.6],
                    ['False', 0.4]])
    # INTELIGENCIA
    F1 = Factor("31", [Intelligence])
    F1.add_values([['True',  0.7],
                    ['False', 0.3]])
    
    # NOTA
    F2 = Factor("F2", [Difficulty, Intelligence, Grade])
    F2.add_values([['True', 'True', 'A', .3],
                    ['True', 'True', 'B', .4],
                    ['True', 'True', 'C', .3],
                    ['True', 'False', 'A', .9],
                    ['True', 'False', 'B', .08],
                    ['True', 'False', 'C', .02],
                    ['False', 'True', 'A', .05],
                    ['False', 'True', 'B', .25],
                    ['False', 'True', 'C', .7],                    
                    ['False', 'False', 'A', .5],
                    ['False', 'False', 'B', .3],
                    ['False', 'False', 'C', .2]])

    # SAT
    F3 = Factor("F3", [Intelligence, SAT])
    F3.add_values([['True', 'True', .95],
                    ['True', 'False', .05],
                    ['False', 'True', .2],
                    ['False', 'False', .8]])

    # CARTA
    F4 = Factor("F4", [Grade, Letter])
    F4.add_values([['A', 'True', .1],
                    ['A', 'False', .9],
                    ['B', 'True', .4],
                    ['B', 'False', .6],
                    ['C', 'True', .99],
                    ['C', 'False', .01]])

    StudentNet = BN("student usa", [Difficulty, Intelligence, Grade, SAT, Letter], [F0, F1, F2, F3, F4])
    
    # preuba de variable de aliminacion Variable de Eliminacion
    # PRUEBA STUDENT USA VARIABLE DE ELIMINACION
    #-------------------------------------------------------------------------------
    
    preuba1 = VETest(StudentNet,[[Difficulty, 'True'], [SAT, 'True']],Letter,[[0.4584, 0.5416]], 5, "VARIABLE DE ELIMINACION QUERY 1")
    
    # guardar los valores en entrada y salida
    # y acumular para mostrar la salida
    
    entrada = 0
    salida = 0     
    inp,out = preuba1.test()    
    entrada += inp
    salida += out
    # VERIFICAR LA COMPARACION
    if( entrada == salida):
        print("")
        print("TODOS LOS VARS COENCIDENTES: {} de {}".format(entrada, salida))
    else:
        print("")
        print("NO COENCIDENTES LOS VARS: {} de {}".format(entrada, salida))


EJECUTAR PRUEBA : VARIABLE DE ELIMINACION QUERY 1
-----------------------------------------------------------

Valor SamIam [F,T]: [0.4584, 0.5416]
Valor Implementacion [F,T]:  [0.45843172413793104, 0.541568275862069]

TODOS LOS VARS COENCIDENTES: 5 de 5


finalmente podemos decir que  que los resultados mostrados en la implementacion para esta consulta son los correctos 